# BIBLIOTECA

In [7]:
pip install certifi


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install requests


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip-system-certs


Usage:   
  /Library/Frameworks/Python.framework/Versions/3.13/bin/python3.13 -m pip <command> [options]

no such option: -s
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import ssl
import requests
import certifi
from io import StringIO
import datetime
import time 

# WEB SCRAPING BCB ORIGINAL

In [11]:
ssl._create_default_https_context = ssl._create_unverified_context

# URL da tabela HTML
url = "https://olinda.bcb.gov.br/olinda/servico/leiloes_selic/versao/v1/odata/leiloesTitulosPublicos(dataMovimentoInicio=@dataMovimentoInicio,dataMovimentoFim=@dataMovimentoFim,dataLiquidacao=@dataLiquidacao,codigoTitulo=@codigoTitulo,dataVencimento=@dataVencimento,edital=@edital,tipoPublico=@tipoPublico,tipoOferta=@tipoOferta)?$top=100&$format=text/html"

# Leitura da tabela HTML usando pandas
tabela = pd.read_html(url)[0]  # [0] para selecionar a primeira tabela da página, se houver várias.

# Agora você tem a tabela em um DataFrame pandas chamado "tabela"
# Para ver as primeiras linhas da tabela:
tabela.head()

,id,dataMovimento,dataLiquidacao,edital,tipoPublico,prazo,quantidadeOfertada,quantidadeAceita,codigoTitulo,dataVencimento,tipoOferta,ofertante,quantidadeOfertadaSegundaRodada,quantidadeAceitaSegundaRodada,cotacaoMedia,cotacaoCorte,taxaMedia,taxaCorte,financeiro
0,9fe040c34a7cb2deef014f780002500,2025-02-20 00:00:00,2025-02-21 00:00:00,41,TodoMercado,222,2000000,770000,100000,2025-10-01 00:00:00,Venda,Tesouro Nacional,0.0,0.0,921885722,921811562,144357,14451,7099
1,9fe040c34a7cb2deef014f78000256c,2025-02-20 00:00:00,2025-02-21 00:00:00,42,TodoMercado,3601,4000000,4000000,950199,2035-01-01 00:00:00,Venda,Tesouro Nacional,1000000.0,NaN,786758827,786371578,146942,147038,3147
2,9fe040c34a7cb2deef014f780002555,2025-02-20 00:00:00,2025-02-21 00:00:00,41,TodoMercado,2505,5000000,5000000,100000,2032-01-01 00:00:00,Venda,Tesouro Nacional,1250000.0,NaN,393954684,393803725,146503,146568,19698
3,9fe040c34a7cb2deef014f78000253f,2025-02-20 00:00:00,2025-02-21 00:00:00,42,TodoMercado,2140,3000000,3000000,950199,2031-01-01 00:00:00,Venda,Tesouro Nacional,750000.0,NaN,844033074,84372226,14733,147428,25321
4,9fe040c34a7cb2deef014f78000252d,2025-02-20 00:00:00,2025-02-21 00:00:00,41,TodoMercado,1410,15000000,15000000,100000,2029-01-01 00:00:00,Venda,Tesouro Nacional,3750000.0,NaN,594545719,594385243,145593,145674,89182


# SELIC - RESULTADOS DOS LEILÕES

In [12]:
import ssl
import pandas as pd
from datetime import datetime

# Desabilitar verificação SSL
ssl._create_default_https_context = ssl._create_unverified_context

# Definir a data de hoje
data_hoje = datetime.now().strftime('%Y-%m-%d')

# URL da tabela HTML com a data de hoje
url = f"https://olinda.bcb.gov.br/olinda/servico/leiloes_selic/versao/v1/odata/leiloesTitulosPublicos(dataMovimentoInicio=@dataMovimentoInicio,dataMovimentoFim=@dataMovimentoFim,dataLiquidacao=@dataLiquidacao,codigoTitulo=@codigoTitulo,dataVencimento=@dataVencimento,edital=@edital,tipoPublico=@tipoPublico,tipoOferta=@tipoOferta)?@dataMovimentoInicio='{data_hoje}'&@dataMovimentoFim='{data_hoje}'&$top=100&$format=text/html"

# Leitura da tabela HTML usando pandas
tabela = pd.read_html(url)[0]

# 1. Reorganizar as colunas
colunas_desejadas = ['edital', 'tipoOferta', 'codigoTitulo', 'ofertante', 'prazo', 'dataLiquidacao','quantidadeOfertada', 'quantidadeAceita', 'taxaCorte']
tabela = tabela[colunas_desejadas]

# 2. Criar a coluna "NomeTitulo"
tabela['NomeTitulo'] = tabela['codigoTitulo'].map({210100: 'LFT 210100', 760199: 'NTN-B 760199'})

# 3. Mover a coluna "NomeTitulo" para a posição correta
colunas = list(tabela.columns)
colunas.insert(colunas.index('codigoTitulo') + 1, colunas.pop(colunas.index('NomeTitulo')))
tabela = tabela[colunas]

# 4. Ordenar por 'edital' com ordem personalizada
ordem_edital = [37, 38, 39]
tabela['edital'] = pd.Categorical(tabela['edital'], categories=ordem_edital, ordered=True)

# 5. Ordenar por 'prazo' com ordem personalizada
ordem_prazo = sorted(tabela['prazo'].unique())
tabela['prazo'] = pd.Categorical(tabela['prazo'], categories=ordem_prazo, ordered=True)

# 6. Criar a coluna "AnoTitulo"
tabela['AnoTitulo'] = (datetime.now() + pd.to_timedelta(pd.to_numeric(tabela['prazo'], errors='coerce'), unit='D')).dt.year

# 7. Mover a coluna "AnoTitulo" para a posição correta
colunas = list(tabela.columns)
colunas.insert(colunas.index('NomeTitulo') + 1, colunas.pop(colunas.index('AnoTitulo')))
tabela = tabela[colunas]

# 8. Ordenar o DataFrame por 'edital' e 'prazo'
tabela_ordenada = tabela.sort_values(by=['edital', 'prazo'])

# 9. Formatar a coluna "taxaCorte" como porcentagem
tabela_ordenada['taxaCorte'] = tabela_ordenada['taxaCorte'] / 10000
tabela_ordenada['taxaCorte'] = tabela_ordenada['taxaCorte'].astype(str) + '%'

# 10. Formatar a coluna "quantidadeAceita"
tabela_ordenada['quantidadeAceita'] = tabela_ordenada['quantidadeAceita'].apply(lambda x: f'{x:,.0f}'.replace(',', '.'))

# 11. Formatar a coluna "quantidadeOfertada"
tabela_ordenada['quantidadeOfertada'] = tabela_ordenada['quantidadeOfertada'].apply(lambda x: f'{x:,.0f}'.replace(',', '.'))

# Visualizar as primeiras linhas da tabela:
tabela_ordenada.head()

,edital,tipoOferta,codigoTitulo,NomeTitulo,AnoTitulo,ofertante,prazo,dataLiquidacao,quantidadeOfertada,quantidadeAceita,taxaCorte
0,NaN,Venda,100000,NaN,2025,Tesouro Nacional,222,2025-02-21 00:00:00,2.000.000,770.000,1.4451%
1,NaN,Venda,100000,NaN,2027,Tesouro Nacional,769,2025-02-21 00:00:00,3.000.000,3.000.000,1.4629%
2,NaN,Venda,100000,NaN,2028,Tesouro Nacional,1410,2025-02-21 00:00:00,15.000.000,15.000.000,14.5674%
3,NaN,Venda,950199,NaN,2030,Tesouro Nacional,2140,2025-02-21 00:00:00,3.000.000,3.000.000,14.7428%
4,NaN,Venda,100000,NaN,2031,Tesouro Nacional,2505,2025-02-21 00:00:00,5.000.000,5.000.000,14.6568%
